In [ ]:
import cv2
import numpy as np
import os

# 이미지가 들어있는 루트 폴더 경로
root_folder = r'C:\Users\1102w\Downloads\cropped_images\cropped_images'

# 하위 폴더를 반복하며 이미지 파일들을 처리
for subdir, dirs, files in os.walk(root_folder):
    for file in files:
        # 이미지 파일 경로 생성
        file_path = os.path.join(subdir, file)
        
        # 이미지 불러오기
        image = cv2.imread(file_path)
        
        if image is None:
            continue  # 이미지가 로드되지 않으면 다음 파일로 넘어감

        # 이미지를 그레이스케일로 변환
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # 경계선 좌표를 저장할 리스트
        col_boundaries = []

        # 열 단위로 경계선을 찾습니다
        previous_col_mean = np.mean(gray_image[:, 0])
        for i in range(1, gray_image.shape[1]):  # 첫 번째 열을 제외하고 반복
            current_col_mean = np.mean(gray_image[:, i])

            if previous_col_mean == 255 and current_col_mean < 255:
                col_boundaries.append(i)  # 경계선을 찾으면 좌표를 저장

            previous_col_mean = current_col_mean

        # 마지막 열의 좌표를 경계선으로 추가
        col_boundaries.append(gray_image.shape[1])

        # 열 경계선 사이의 거리를 계산하여 일정 거리 이상일 때 잘라서 저장
        for k in range(1, len(col_boundaries)):
            if col_boundaries[k] - col_boundaries[k-1] > 100:
                cropped_image = image[:, col_boundaries[k-1]:col_boundaries[k]]
                
                # 원래 파일 이름을 유지하며 저장할 파일 이름 생성
                base_name = os.path.splitext(file)[0]
                extension = os.path.splitext(file)[1]
                save_path = os.path.join(subdir, f'{base_name}_part{k}{extension}')
                
                # 이미지 저장
                cv2.imwrite(save_path, cropped_image)

print("이미지 처리 완료!")
